## `data`

In [176]:
import subprocess
import psycopg2
import pandas as pd

from sklearn.metrics import classification_report

In [2]:

creds = {'dbname': 'd1mikus7g5uss8',
 'host': 'ec2-99-81-68-240.eu-west-1.compute.amazonaws.com',
 'port': '5432',
 'user': 'iohznziolcottb',
 'password': '5a812ea29f6142328bc2afab03e48e6462939babe87610342cdf12e2d357a4f0',
 'sslmode': 'require'}

creds

{'dbname': 'd1mikus7g5uss8',
 'host': 'ec2-99-81-68-240.eu-west-1.compute.amazonaws.com',
 'port': '5432',
 'user': 'iohznziolcottb',
 'password': '5a812ea29f6142328bc2afab03e48e6462939babe87610342cdf12e2d357a4f0',
 'sslmode': 'require'}

In [3]:
conn = psycopg2.connect(host=creds['host'], database=creds['dbname'],
                                user=creds['user'],
                                password=creds['password'])
cur = conn.cursor()

In [4]:
select_Query = 'select * from public."DiscH_prototype_question"'
cur.execute(select_Query)
questions = cur.fetchall()
question_col = ["question_id", "description", "num_response", "label_in_question"]
questions_df = pd.DataFrame(questions, columns=question_col)

select_Query = 'select * from public."DiscH_prototype_answer"'
cur.execute(select_Query)
answers = cur.fetchall()
answer_col = ["answer_id", "answer_category_num", "answer_justification", "answer_upvote",
              "account_id_id", "question_id_id", "date"]
answers_df = pd.DataFrame(answers, columns=answer_col)

select_Query = 'select * from public."DiscH_prototype_answer_bow"'
cur.execute(select_Query)
answers_bow = cur.fetchall()
answer_bow_col = ["answer_id", "answer_category_num", "answer_justification", "answer_upvote",
              "account_id_id", "question_id_id", "answer_text_comment", "date"]
answers_bow_df = pd.DataFrame(answers_bow, columns=answer_bow_col)


select_Query = 'select * from public."DiscH_prototype_achievement"'
cur.execute(select_Query)
achievements = cur.fetchall()
achievement_col = ["achievement_id", "achievement_type", "achievement_date", "account_id", "question_id_id", "value"]
achievements_df = pd.DataFrame(achievements, columns=achievement_col)

In [5]:
select_Query = 'select q.description, a.answer_category_num, a.answer_justification, a.answer_upvote, \
              a.account_id_id from public."DiscH_prototype_question" q RIGHT JOIN public."DiscH_prototype_answer" a \
ON a.question_id_id=q.question_id'
cur.execute(select_Query)
QAs = cur.fetchall() 
QA_col = ["question_description", "answer_category_num", "answer_justification", "answer_upvote", "account_id"]
QA_df = pd.DataFrame(QAs, columns=QA_col)

In [94]:
# response_500 = pd.read_excel("../data/responses_data_500.xlsx")

In [243]:
clean_data = QA_df.copy()
# HERE
response_500 = pd.read_excel("../data/responses_data_500.xlsx")
clean_data = response_500.copy().rename(columns={"description":'question_description'})

clean_data  = clean_data[['question_description', 'answer_category_num']]


In [244]:
clean_data.drop_duplicates(subset='question_description', inplace=True)
clean_data.reset_index(drop=True, inplace=True)

In [245]:
# checking how many comments have different votes
for i, row in clean_data.iterrows():

    comment = row.question_description

    # getting the dataframe for that comment
    temp_df = clean_data[clean_data.question_description == comment]
    
    # how many unique answers that comment has
    n_ = temp_df.answer_category_num.nunique()

    # changing the answer_category_num to the most frequent one
    if n_ >= 2:
        most_voted_label = temp_df.answer_category_num.value_counts().index[0]
        clean_data.at[i, 'answer_category_num'] = most_voted_label

In [246]:
clean_data.shape

(374, 2)

## `1) TF LSTM model`

In [247]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
%matplotlib inline

In [248]:
X = clean_data.question_description
Y = clean_data.answer_category_num

le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [249]:
# row 0
X[0],  Y[0]

('وزير الخارجية اللبناني جبران باسيل قال في سلسلة تغريداته عقب اختتام القمة العربية التنموية الاقتصادية والاجتماعية لمسنا تج…',
 array([3]))

In [250]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.15)

In [251]:
max_words = 2000
max_len = 200
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [252]:
from keras.utils import np_utils

In [253]:
# we need to binarize the labels for the neural net
ytrain_enc = np_utils.to_categorical(Y_train)
ytest_enc = np_utils.to_categorical(Y_test)

In [254]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(128)(layer)
    layer = Dense(256, name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dense(6, name='out_layer')(layer)
    layer = Activation(activation='softmax')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [255]:
model = RNN()
model.summary()
model.compile(loss='categorical_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 200)]             0         
                                                                 
 embedding_14 (Embedding)    (None, 200, 50)           100000    
                                                                 
 lstm_14 (LSTM)              (None, 128)               91648     
                                                                 
 FC1 (Dense)                 (None, 256)               33024     
                                                                 
 activation_28 (Activation)  (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 6)                 1542      
                                                                 
 activation_29 (Activation)  (None, 6)                 0  

In [256]:
model.fit(sequences_matrix, ytrain_enc, batch_size=256, epochs=25, callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

# model.fit(sequences_matrix, ytrain_enc, batch_size=256, epochs=25, validation_split=0.2)

Epoch 1/25
1/2 [==============>...............] - ETA: 1s - loss: 1.7923 - accuracy: 0.0938WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy


2/2 [==============================] - 2s 28ms/step - loss: 1.7912 - accuracy: 0.1293
Epoch 2/25
1/2 [==============>...............] - ETA: 0s - loss: 1.7597 - accuracy: 0.3203WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy


2/2 [==============================] - 0s 29ms/step - loss: 1.7715 - accuracy: 0.3060
Epoch 3/25
1/2 [==============>...............] - ETA: 0s - loss: 1.7580 - accuracy: 0.2969WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy


2/2 [==============================] - 0s 27ms/step - loss: 1.7549 - accuracy: 0.3060
Epoch 4/25
1/2 [==============>...............] - ETA: 0s - loss: 1.7338 - accuracy: 0.3086WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy


2/2 [==============================] - 0s 26ms/step - loss: 1.7297 - accuracy: 0.3060
Epoch 5/25
1/2 [==============>...............] - ETA: 0s - loss: 1.6912 - accuracy: 0.3320WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy


...

2/2 [==============================] - 0s 20ms/step - loss: 1.9026 - accuracy: 0.4669
Epoch 22/25
1/2 [==============>...............] - ETA: 0s - loss: 0.8799 - accuracy: 0.7812WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy


2/2 [==============================] - 0s 20ms/step - loss: 0.8769 - accuracy: 0.7760
Epoch 23/25
1/2 [==============>...............] - ETA: 0s - loss: 0.7819 - accuracy: 0.8320WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy


2/2 [==============================] - 0s 20ms/step - loss: 0.7883 - accuracy: 0.8360
Epoch 24/25
1/2 [==============>...............] - ETA: 0s - loss: 0.7213 - accuracy: 0.8477WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy


2/2 [==============================] - 0s 23ms/step - loss: 0.7137 - accuracy: 0.8423
Epoch 25/25
1/2 [==============>...............] - ETA: 0s - loss: 0.6440 - accuracy: 0.8633WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy


2/2 [==============================] - 0s 24ms/step - loss: 0.6491 - accuracy: 0.8644


In [257]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [258]:
accr = model.evaluate(test_sequences_matrix, ytest_enc)

2/2 [==============================] - 0s 6ms/step - loss: 1.7289 - accuracy: 0.2807


In [259]:
test_sequences_matrix.shape

(57, 200)

In [260]:
LSTM_predictions = model.predict(test_sequences_matrix)

In [261]:
LSTM_predictions = LSTM_predictions.argmax(axis=1) 

In [262]:
LSTM_predictions

array([1, 1, 0, 3, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 5, 0, 0, 0, 0, 0, 1, 5, 5, 0, 0, 0, 0, 5, 5, 1, 3, 0, 1,
       0, 0, 5, 5, 1, 0, 0, 1, 0, 0, 1, 0, 0], dtype=int64)

In [263]:
print(classification_report(Y_test, LSTM_predictions))

              precision    recall  f1-score   support

           0       0.27      0.62      0.38        16
           1       0.45      0.31      0.37        16
           2       0.00      0.00      0.00         6
           3       0.00      0.00      0.00         4
           4       0.00      0.00      0.00         6
           5       0.14      0.11      0.12         9

    accuracy                           0.28        57
   macro avg       0.14      0.17      0.15        57
weighted avg       0.23      0.28      0.23        57



C:\Users\modaj\miniconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\modaj\miniconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\modaj\miniconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [170]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 1.730
  Accuracy: 0.241


## `2) TF BERT model`

In [264]:
import tokenization
import tensorflow as tf
import tensorflow_hub as hub  
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import numpy as np

In [265]:
# import BERT tokenization

# !wget --quiet https://raw.githubusercontent.com/google-research/bert/master/tokenization.py

In [266]:
clean_data

,question_description,answer_category_num
0,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,Religious affiliation
1,سورية بلد الحضارات تربطها بعلية او بحيوان,Racist
2,اخي الحاج اذا شعرت انك محرجا من الانتقادات لتص...,Normal
3,ما فيك تعيش بلا ما تكب فتن ليل نهار وبكرة قلهم...,Normal
4,هذا البطل الذي قاتل وجاذف بحياته لتحيا انت يا ...,Religious affiliation
...,...,...
369,ملعون هو وابوه وابو عيلتو وابو اسرائيل على ابو...,Mockery
370,عاهاتك ياوطن,Mockery
371,أمر عمليات اليوم جبران باسيل ينفذ اوامر الولاي...,Normal
372,حط صرماية بتمك وتم يلي خلفك فهمت ولاك كلب,Violent


In [267]:
# X, y = clean_data[["question_description"]], clean_data["answer_category_num"]

train_data, test_data = train_test_split(clean_data, test_size=0.15, random_state=42)

In [268]:
train_data.shape, test_data.shape

((317, 2), (57, 2))

In [269]:
label = preprocessing.LabelEncoder()
y = label.fit_transform(train_data['answer_category_num'])
y = to_categorical(y)
y_test = label.transform(test_data['answer_category_num'])
y

array([[0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0.]], dtype=float32)

#### create a BERT embedding layer by importing the BERT model from hub.KerasLayer

In [270]:
m_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'
bert_layer = hub.KerasLayer(m_url, trainable=True)

create a BERT vocab_file in the form a numpy array. We then set the text to lowercase and finally we pass our vocab_file and do_lower_case variables to the Tokenizer object.

In [271]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
        
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len-len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence) + [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
        
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [272]:
def build_model(bert_layer, max_len=512):
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")
    
    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    
    clf_output = sequence_output[:, 0, :]
    
    lay = tf.keras.layers.Dense(64, activation='relu')(clf_output)
    lay = tf.keras.layers.Dropout(0.2)(lay)
    lay = tf.keras.layers.Dense(32, activation='relu')(lay)
    lay = tf.keras.layers.Dropout(0.2)(lay)
    out = tf.keras.layers.Dense(6, activation='softmax')(lay)
    
    model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(tf.keras.optimizers.Adam(lr=2e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [273]:
max_len = 250
train_input = bert_encode(train_data.question_description.values, tokenizer, max_len=max_len)
test_input = bert_encode(test_data.question_description.values, tokenizer, max_len=max_len)
train_labels = y
test_labels = y_test

In [274]:
labels = label.classes_
print(labels)

['Mockery' 'Normal' 'Racist' 'Religious affiliation' 'Sexual harrasment'
 'Violent']


In [275]:
model = build_model(bert_layer, max_len=max_len)
model.summary()

Model: "model_16"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 250)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 250)]        0           []                               
                                                                                                  
 segment_ids (InputLayer)       [(None, 250)]        0           []                               
                                                                                                  
 keras_layer_1 (KerasLayer)     [(None, 768),        109482241   ['input_word_ids[0][0]',         
                                 (None, 250, 768)]                'input_mask[0][0]',      

C:\Users\modaj\miniconda3\envs\tf\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [276]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)

train_sh = model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=7,
    callbacks=[checkpoint, earlystopping],
    batch_size=3,
    verbose=1
)

Epoch 1/7
85/85 [==============================] - ETA: 0s - loss: 1.7497 - accuracy: 0.2964
Epoch 1: val_accuracy improved from -inf to 0.25000, saving model to model.h5
85/85 [==============================] - 36s 299ms/step - loss: 1.7497 - accuracy: 0.2964 - val_loss: 1.7428 - val_accuracy: 0.2500
Epoch 2/7
85/85 [==============================] - ETA: 0s - loss: 1.7012 - accuracy: 0.3162
Epoch 2: val_accuracy did not improve from 0.25000
85/85 [==============================] - 22s 255ms/step - loss: 1.7012 - accuracy: 0.3162 - val_loss: 1.7827 - val_accuracy: 0.2344
Epoch 3/7
85/85 [==============================] - ETA: 0s - loss: 1.7922 - accuracy: 0.2648
Epoch 3: val_accuracy did not improve from 0.25000
85/85 [==============================] - 21s 253ms/step - loss: 1.7922 - accuracy: 0.2648 - val_loss: 1.7825 - val_accuracy: 0.2500
Epoch 4/7
85/85 [==============================] - ETA: 0s - loss: 1.7467 - accuracy: 0.2727
Epoch 4: val_accuracy did not improve from 0.25000
8

In [281]:
max_len = 250
test_input = bert_encode(test_data.question_description.values, tokenizer, max_len=max_len)

BERT_predictions = model.predict(test_input)

In [282]:
BERT_predictions = BERT_predictions.argmax(axis=1) 

In [283]:
BERT_predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [284]:
y_test

array([5, 0, 3, 5, 0, 0, 5, 4, 5, 4, 1, 2, 3, 3, 5, 3, 3, 5, 1, 0, 1, 1,
       1, 2, 1, 5, 5, 5, 0, 0, 3, 4, 0, 2, 5, 1, 4, 3, 0, 1, 3, 2, 4, 0,
       0, 5, 2, 0, 3, 5, 4, 4, 5, 0, 5, 0, 1])

In [285]:
print(classification_report(y_test, BERT_predictions))

              precision    recall  f1-score   support

           0       0.23      1.00      0.37        13
           1       0.00      0.00      0.00         9
           2       0.00      0.00      0.00         5
           3       0.00      0.00      0.00         9
           4       0.00      0.00      0.00         7
           5       0.00      0.00      0.00        14

    accuracy                           0.23        57
   macro avg       0.04      0.17      0.06        57
weighted avg       0.05      0.23      0.08        57



C:\Users\modaj\miniconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\modaj\miniconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\modaj\miniconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## `3) ML models`

In [223]:
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import accuracy_score, classification_report

In [224]:
train_data, test_data = train_test_split(clean_data, test_size=0.15, random_state=42)

In [225]:
train_data

,question_description,answer_category_num
2550,سمّعنا سكوتك يا بغلول,Mockery
108,بدك شي شاكوش يضرب راسك ليفتح مخك السيد مش لاكش...,Mockery
1206,كتير قوية,Mockery
2416,هو يقصد خيانه الفلسطينين للكويت الدوله الوحيده...,Religious affiliation
2441,طب خود هالكلمتين كول هوا,Violent
...,...,...
1638,خصوصا ان سي ان ان طردت,Sexual harrasment
1095,يعدمني ياك يا زيهوفر الحيوان,Normal
1130,طز بهيك عرب طز بهيك دولة تفو,Sexual harrasment
1294,ربنا معاكي يافلسطين أخوكم من مصر,Normal


In [226]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1))
tfidf_tr = tfidf_vectorizer.fit_transform(train_data['question_description'])
tfidf_val = tfidf_vectorizer.transform(test_data['question_description'])

In [227]:
# Model definitions
LogReg_model = LogisticRegression()
RandomForestClassifier_model = RandomForestClassifier(max_depth=3, random_state=0)

MultinomialNB_model = MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)
SGDClassifier_model = SGDClassifier(class_weight='balanced', penalty='l1')
KNeighborsClassifier_model = KNeighborsClassifier(n_neighbors=3)
DecisionTreeClassifier_model = DecisionTreeClassifier(random_state=0)

models = [LogReg_model, RandomForestClassifier_model, DecisionTreeClassifier_model,
          SGDClassifier_model,   
          KNeighborsClassifier_model,  MultinomialNB_model]
model_names = ['LogisticRegression', 'RandomForestClassifier', 'DecisionTreeClassifier',
               'SGDClassifier', 'KNeighborsClassifier', 'MultinomialNB']

In [228]:
X_train, y_train = tfidf_tr, train_data['answer_category_num']
X_train.shape, y_train.shape

((2628, 13328), (2628,))

In [229]:
X_test, y_test = tfidf_val, test_data['answer_category_num']
X_test.shape, y_test.shape

((464, 13328), (464,))

In [230]:
def train_models(X_tr, X_te, y_tr, y_te):
    for i, model in enumerate(models):
        print(f"Model: {model_names[i]}")
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_te)
        print('val accuracy %s' % accuracy_score(y_te, y_pred))
        print(classification_report(y_te, y_pred))  # for further evaluation
        
        print()

In [231]:
train_models(X_train, X_test, y_train, y_test)

Model: LogisticRegression
val accuracy 0.3103448275862069
                       precision    recall  f1-score   support

              Mockery       0.27      0.46      0.34       106
               Normal       0.38      0.55      0.45       128
               Racist       0.25      0.02      0.04        47
Religious affiliation       0.20      0.02      0.03        57
    Sexual harrasment       0.33      0.02      0.04        42
              Violent       0.27      0.25      0.26        84

             accuracy                           0.31       464
            macro avg       0.28      0.22      0.19       464
         weighted avg       0.29      0.31      0.26       464


Model: RandomForestClassifier
val accuracy 0.26939655172413796
                       precision    recall  f1-score   support

              Mockery       0.25      0.67      0.36       106
               Normal       0.31      0.42      0.35       128
               Racist       0.00      0.00      0.00   

C:\Users\modaj\miniconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\modaj\miniconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\modaj\miniconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


val accuracy 0.25
                       precision    recall  f1-score   support

              Mockery       0.26      0.39      0.31       106
               Normal       0.32      0.29      0.30       128
               Racist       0.18      0.19      0.19        47
Religious affiliation       0.23      0.12      0.16        57
    Sexual harrasment       0.15      0.14      0.15        42
              Violent       0.23      0.19      0.21        84

             accuracy                           0.25       464
            macro avg       0.23      0.22      0.22       464
         weighted avg       0.25      0.25      0.24       464


Model: SGDClassifier
val accuracy 0.25
                       precision    recall  f1-score   support

              Mockery       0.32      0.31      0.32       106
               Normal       0.35      0.33      0.34       128
               Racist       0.19      0.21      0.20        47
Religious affiliation       0.18      0.21      0.19    

## `4) Light GBM`

In [286]:
import lightgbm as lgb

In [287]:
clean_data

,question_description,answer_category_num
0,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,Religious affiliation
1,سورية بلد الحضارات تربطها بعلية او بحيوان,Racist
2,اخي الحاج اذا شعرت انك محرجا من الانتقادات لتص...,Normal
3,ما فيك تعيش بلا ما تكب فتن ليل نهار وبكرة قلهم...,Normal
4,هذا البطل الذي قاتل وجاذف بحياته لتحيا انت يا ...,Religious affiliation
...,...,...
369,ملعون هو وابوه وابو عيلتو وابو اسرائيل على ابو...,Mockery
370,عاهاتك ياوطن,Mockery
371,أمر عمليات اليوم جبران باسيل ينفذ اوامر الولاي...,Normal
372,حط صرماية بتمك وتم يلي خلفك فهمت ولاك كلب,Violent


In [288]:
train_data, test_data = train_test_split(clean_data, test_size=0.15, random_state=42)

In [289]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1))
tfidf_tr = tfidf_vectorizer.fit_transform(train_data['question_description'])
tfidf_val = tfidf_vectorizer.transform(test_data['question_description'])

In [290]:
X_train, X_test, y_train, y_test = tfidf_tr, tfidf_val, train_data["answer_category_num"], test_data["answer_category_num"]

In [291]:
clf = lgb.LGBMClassifier()
clf.fit(X_train, y_train)

LGBMClassifier()

In [292]:
y_pred=clf.predict(X_test)

In [293]:
# view accuracy
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_test, y_pred)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy))

LightGBM Model accuracy score: 0.2632


In [295]:
print(classification_report(y_test, y_pred))  # for further evaluation

                       precision    recall  f1-score   support

              Mockery       0.29      0.77      0.42        13
               Normal       0.30      0.33      0.32         9
               Racist       0.00      0.00      0.00         5
Religious affiliation       0.00      0.00      0.00         9
    Sexual harrasment       0.00      0.00      0.00         7
              Violent       0.50      0.14      0.22        14

             accuracy                           0.26        57
            macro avg       0.18      0.21      0.16        57
         weighted avg       0.24      0.26      0.20        57



### References

- https://www.kaggle.com/code/nayansakhiya/text-classification-using-bert
